# Machine Learning Project GMIT by Greg Feeley

## Describe, Infer and Predict Using Boston Housing Dataset

In [ ]:
#import libraries
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import seaborn as sns

In [ ]:
#import dataset from sklearn and sklearn packages
from sklearn.datasets import load_boston
boston_dataset = load_boston()
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

#assign dataframe and print sample edited from https://anujdutt9.github.io/LinearRegressionTF.html
df = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
print(df.head())           

In [ ]:
df['PRICE'] = boston_dataset.target
#show shape of data for inspection
df.shape

In [ ]:
#use pearson correlation method edited from https://pythontic.com/pandas/dataframe-computations/correlation
df.corr(method='pearson')

In [ ]:
#heatmap of correlation annotate each cell with numeric value 
sns.set(rc={'figure.figsize':(9.7,9.28)})
sns.heatmap(df.corr().round(4), square=True, annot=True)

In [ ]:
#dataframe columns
df.keys()

In [ ]:
#summary of data
df.describe()

In [ ]:
#edited from https://scipy-lectures.org/packages/scikit-learn/auto_examples/plot_boston_prediction.html
#histogram of of column quantities in relation to price
plt.figure(figsize=(4, 3))
plt.hist(boston_dataset.target,color='purple' )
plt.xlabel('price ($1000s)')
plt.ylabel('count')
plt.tight_layout()
#show scatter for each column in relation to price use enumerate to loop
for index, feature_name in enumerate(boston_dataset.feature_names):
    plt.figure(figsize=(4, 3))
    plt.scatter(boston_dataset.data[:, index], boston_dataset.target,color='purple')
    plt.ylabel('Price', size=15)
    plt.xlabel(feature_name, size=15)
    plt.tight_layout()

In [ ]:
#Linear regression
xchas=df.CHAS
yprice=df.PRICE
print(xchas)
print(yprice)

In [ ]:
#boxplots charles river
df.groupby('RM').boxplot(fontsize=24,rot=85,figsize=(22,12),patch_artist=True)